In [ ]:
import os
import dotenv

dotenv.load_dotenv()

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash-latest",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
print(llm.invoke("Hello, how are you?"))

In [ ]:
from langchain.utilities import SQLDatabase

db = SQLDatabase.from_uri(
    f"mysql+pymysql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
)
print(db.table_info)

In [ ]:
from langchain_experimental.sql import SQLDatabaseChain

chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

In [ ]:
qns1 = chain.invoke("How many t-shirts are there?")

In [ ]:
print(qns1)

In [ ]:
db.run("SELECT COUNT(*) FROM t_shirts")

In [ ]:
#! Wrong answer as it should multiply by the quantity
qns1 = chain.run(
    "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
)

In [ ]:
qns1

In [ ]:
qns2 = chain.run("How much is the price of the inventory for all small size t-shirts?")

In [ ]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = chain.run(sql_code)

In [ ]:
qns4 = chain.run("How many white color Levi's t shirts we have available?")

In [ ]:
qns1 = chain.run(
    "How many t-shirts do we have left for nike in extra small size and white color?"
)
qns2 = chain.run("How much is the price of the inventory for all small size t-shirts?")
qns2 = chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")
qns3 = chain.run(
    "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"
)
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = chain.run(sql_code)
qns4 = chain.run(
    "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'"
)
qns5 = chain.run("How many white color Levi's t shirts we have available?")
qns5 = chain.run(
    "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'"
)


In [ ]:
print(qns1)
print(qns2)
print(qns3)
print(qns4)
print(qns5)

In [ ]:
few_shots = [
    {
        "Question": "How many t-shirts do we have left for Nike in XS size and white color?",
        "SQLQuery": "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
        "SQLResult": "Result of the SQL query",
        "Answer": qns1,
    },
    {
        "Question": "How much is the total price of the inventory for all S-size t-shirts?",
        "SQLQuery": "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
        "SQLResult": "Result of the SQL query",
        "Answer": qns2,
    },
    {
        "Question": "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?",
        "SQLQuery": """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
        "SQLResult": "Result of the SQL query",
        "Answer": qns3,
    },
    {
        "Question": "If we have to sell all the Levi’s T-shirts today. How much revenue our store will generate without discount?",
        "SQLQuery": "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
        "SQLResult": "Result of the SQL query",
        "Answer": qns4,
    },
    {
        "Question": "How many white color Levi's shirt I have?",
        "SQLQuery": "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
        "SQLResult": "Result of the SQL query",
        "Answer": qns5,
    },
]


In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embedding_encoder = GoogleGenerativeAIEmbeddings(model="models/embedding-001")


In [ ]:
to_vectorize =[" ".join(qn.values()) for qn in few_shots]

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_texts(to_vectorize, embedding_encoder, metadatas=few_shots)

In [ ]:
from langchain.prompts import SemanticSimilarityExampleSelector

selector = SemanticSimilarityExampleSelector(vectorstore=vectorstore, k=2)
selector.select_examples({'Question':"How many t-shirts do we have left for Nike in XS size and white color?"})


In [ ]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt

print(_mysql_prompt)
print(PROMPT_SUFFIX)

In [ ]:
from langchain.prompts.prompt import  PromptTemplate

prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="Question: {Question}\nSQL Query: {SQLQuery}\nSQL Result: {SQLResult}\nAnswer: {Answer}\n",
)

In [ ]:
from langchain.prompts import FewShotPromptTemplate

few_shot_prompt = FewShotPromptTemplate(
    example_selector=selector,
    example_prompt=prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input","table_info","top_k"],
)

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [ ]:
db_chain.run("How much is the total price of the inventory for all small t-shirts?")

In [ ]:
db_chain.run("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
db_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")
